# スクレイピング

In [11]:
#マウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#Base64をデコードして画像に戻す関数
import base64
import requests
from bs4 import BeautifulSoup
import os
import re
import time

In [13]:
#Seleniumでは要素を読み終えてからのHTMLを取得する←これをやらないとimgをちゃんととってこない
# Seleniumを使用するのに必要なライブラリをダウンロード
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

from selenium import webdriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [14]:
#WebブラウザをUIなしで扱うための設定
# ドライバーをUIなしで使用するための設定
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--lang=ja')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)

In [15]:
#Seleniumを使ってすべての要素が読み込まれるまで待つという設定を行う
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# ページ上のすべての要素が読み込まれるまで待機（15秒でタイムアウト判定）
WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located)

<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [16]:
#画像をダウンロードする関数
def download_image(url, file_path):
  r = requests.get(url, stream=True)

  if r.status_code == 200:
    with open(file_path, "wb") as f:
      f.write(r.content)
def save_base64_image(data, file_path):
  img = base64.b64decode(data.encode())
  with open(file_path, "wb") as f:
      f.write(img)

In [18]:
#検索list
title_list = [
    "トゥワイス ダヒョン",
    "トゥワイス ジヒョ",
    "トゥワイス ナヨン",
    "トゥワイス モモ",
    "トゥワイス ナヨン",
    "トゥワイス ツゥイ",
    "トゥワイス サナ",
    "トゥワイス ミナ",
    "トゥワイス チェヨン",
    "niziu まゆか",
    "niziu リマ",
    "nizu miihi",
    "nizu riku",
    "nizu nina",
    "nizu maya",
    "nizu mako",
    "Lisa BlackPink",
    "Black Pink Rose",
    "Black Pink Jisoo",
    "Ariana Grande　face",
    "橋本環奈",
    "橋本環奈　顔　かわいい",
    "本田翼　顔　かわいい",
    "本田翼",
    "堀北真希",
    "堀北真希　顔　かわいい",
    "twice NAYEON face",
    "twice JEONGYEON face",
    "twice MOMO face",
    "twice SANA face",
    "twice JIHYO face",
    "twice MINA face",
    "twice DAHYUN face",
    "twice CHAEYOUNG face",
    "twice TZUYU face",
    "ITZY リュジン 顔",
    "ITZY ユナ 顔",
    "ITZY イェジ 顔",
    "ITZY イア 顔",
    "ITZY チェリョン 顔",
    "乃木坂46 齋藤飛鳥　顔",
    "乃木坂46 白石麻衣　顔",
    "乃木坂46 橋本奈々未　顔",
    "乃木坂46 生田絵梨花　顔",
    "乃木坂46 松村沙友理　顔",
    "乃木坂46 深川麻衣　顔",
    "乃木坂46 高山一実　顔",
    "乃木坂46 星野みなみ　顔",
    "乃木坂46 生駒里奈　顔",
    "乃木坂46 秋元真夏　顔",
    "乃木坂46 山下美月　顔",
    "乃木坂46 賀喜遥香　顔",
    "石原さとみ　顔",
    "今田美桜　顔",
    "新木優子　顔",
    "小松菜奈　顔",
    "AKB48 柏木由紀　顔",
    "柏木由紀　顔",
    "小嶋陽菜　顔",
    "西野七瀬　顔",
    "三吉彩花　顔",
    "馬場ふみか　顔",
    "えなこ　顔",
    "NiziU　ミイヒ　顔",
    "山本彩　顔",
    "渡辺美優紀　顔",
    "生田絵梨花　顔",
    "松村沙友理　顔",
    "ももいろクローバーZ 百田夏菜子　顔",
    "忽那汐里　顔",
    "内田理央　顔",
    "AKB48 岡田奈々　顔",
    "山口真帆　顔",
    "三津谷葉子　顔",
    "乃木坂46 中西アルノ　顔",
    "乃木坂46 小川彩　顔",
    "日向坂46 齊藤京子　顔",
    "櫻坂46 守屋麗奈　顔",
    "LE SSERAFIM カズハ　顔",
    "IVE レイ　顔",
    "篠崎愛　顔",
    "加藤美南　顔",
    "加藤玲奈　顔",
    "大和田南那　顔",
    "ダヒョン 顔　かわいい",
    "ジヒョ　顔　かわいい",
    "ナヨン　顔　かわいい",
    "モモ　顔　かわいい",
    "ナヨン　顔　かわいい",
    "ツゥイ　顔　かわいい",
    "サナ　顔　かわいい",
    "ミナ　顔　かわいい",
    "チェヨン　顔　かわいい",
    "まゆか　顔　かわいい　nizu",
    "リマ　顔　かわいい　nizu",
    "ミイヒ 顔　かわいい　nizu",
    "リク　顔　かわいい　nizu",
    "ニナ　顔　かわいい　nizu",
    "マヤ　顔　かわいい　nizu",
    "マコ　顔　かわいい　nizu",
    "齋藤飛鳥　かわいい",
    "白石麻衣　かわいい",
    "橋本奈々未　かわいい",
    "生田絵梨花　かわいい",
    "松村沙友理　かわいい",
    "深川麻衣　かわいい",
    "高山一実　かわいい",
    "星野みなみ　かわいい",
    "生駒里奈　かわいい",
    "秋元真夏　かわいい",
    "山下美月　かわいい",
    "賀喜遥香　かわいい",
    "石原さとみ　かわいい",
    "今田美桜　かわいい",
    "新木優子　かわいい",
    "小松菜奈　かわいい",
    "AKB48 柏木由紀　かわいい",
    "リュジン　かわいい　ITZY",
    "ユナ　かわいい　ITZY",
    "イェジ　かわいい　ITZY",
    "イア　かわいい　ITZY",
    "チェリョン かわいい　ITZY",
    "えなこ　かわいい",
    "佐々木希",
    "佐々木希 顔　かわいい",
    "黒木メイサ",
    "黒木メイサ　顔　かわいい",
    "桐谷美玲",
    "桐谷美玲　顔　かわいい",
    "島崎遥香　顔　かわいい",
    "島崎遥香",
    "Niki　丹羽仁希",
    "Niki/丹羽仁希　顔　かわいい",
    "岡田奈々　顔　かわいい",
    "篠崎愛　顔　かわいい",
    "森小雪　PURPLE K!SS",
    "森小雪　顔　かわいい",
    "ペ・スジ　顔　かわいい",
    "ライサ・アンドリアナ",
    "ライサ・アンドリアナ　顔　かわいい",
    "ペ・スジ",
    "ザラ・ラーソン　顔",
    "アンジェリーナ・ジョリー　顔　美人",
    "ジュー・ジンイー　顔",
    "ケンダル・ジェンナー　顔　美人",
    "テイラー・スウィフト　顔",
    "クロエ・グレース・モレッツ　顔",
    "カミーラ・ベル　顔",
    "ゴルシフテ・ファラハニ　顔",
    "アデレード・ケイン　顔",
    "ジャーダン・ダン　顔",
    "ナナ　AFTERSCHOOL　顔　美人",
    "サラ・ガドン　顔",
    "ナイトレイ　顔　美人",
    "オードリー・ヘップバーン　顔",
    "ティレーヌ・レナ＝ローズ・ブロンドー　顔",
    "キョンギョル　顔",
    "カーラ・デルヴィーニュ　顔",
    "エマ・ワトソン　顔"
]

In [19]:
sheet=1
for title in title_list:
  url = "https://www.bing.com/images/search?q={}".format(title)
  driver.get(url)
  html = driver.page_source.encode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  #soup
  img_tags = soup.find_all("img")
  time.sleep(1)
  img_urls = []
  for img_tag in img_tags:
    alt = img_tag.get("alt")
    if alt == "Image result for "+ title:
      url = img_tag.get("data-src")
      if url is None:
        url = img_tag.get("src")
      if url is not None:
        img_urls.append(url)
  save_dir = "/content/drive/MyDrive/プロダクト演習/picture"
  for index, url in enumerate(img_urls):
    # enumerateを使ってリストのindexを取得。このindexをそのままファイル名にする
    # formatを使えば文字列内の指定した場所に変数の内容を入れることができる
    file_name = "{}.jpg".format(sheet)
    sheet += 1
    """
    os.path.joinを使えば引数に指定した文字列をパスの形に繋げる(joinする）ことができます
    例: os.path.join("A", "B")は「A/B」を出力します
    """
    image_path = os.path.join(save_dir, file_name)
  #base64の接頭辞削除
    if len(re.findall("data:image/jpeg;base64", url)) > 0:
      url = url.replace("data:image/jpeg;base64", "")
      save_base64_image(data=url, file_path=image_path)
    else:
      download_image(url=url, file_path=image_path)
  time.sleep(1)
  #sheet += 1

In [ ]:
print(sheet)

141
